In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow
from  keras.models import Sequential, Model
from  keras.layers import Input,Concatenate, Dense, LeakyReLU,Input, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from  keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from  keras.applications.vgg16 import VGG16, preprocess_input
#from  keras.applications import ResNet152V2,ResNet101V2,ResNet50V2 
#from  keras.applications.resnet_v2 import preprocess_input 

from  keras.optimizers import RMSprop, Adam
from  keras.preprocessing.image import load_img, ImageDataGenerator
from  keras.utils import multi_gpu_model
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw,ImageFont

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras.backend as K
import numpy as np
import random

In [ ]:
def yolf_loss(y_true, y_pred):
    #true e pred sono [32][245]
    
    #tensorflow.print("tensors:", y_pred)
    #tensorflow.print("batch_size:", tensorflow.shape(y_pred)[0])
    
    #y_pred = tensorflow.reshape(y_pred, [tensorflow.shape(y_pred)[0], 7, 7, 5])
    #y_true = tensorflow.reshape(y_true, [tensorflow.shape(y_pred)[0], 7, 7, 5])


    b_p_pred = y_pred[0]
    b_x_pred = y_pred[1]
    b_y_pred = y_pred[2]
    b_w_pred = y_pred[3]
    b_h_pred = y_pred[4]


    b_p = y_true[0]
    b_x = y_true[1]
    b_y = y_true[2]
    b_w = y_true[3]
    b_h = y_true[4]

    #print(b_xy_pred.get_shape(),b_xy.get_shape())
    #print(b_wh_pred.get_shape(),b_wh.get_shape())
    #indicator_coord = K.expand_dims(y_true[ 3], axis=-1) * 1.0
    loss_p = K.sum(K.square(b_p - b_p_pred), axis=-1)
    loss_xy = K.sum(b_p * (K.square(b_x - b_x_pred) + K.square(b_y - b_y_pred)), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    
    #b_w = K.pow(b_w, 0.5)
    #b_h = K.pow(b_h, 0.5)
    #b_w_pred = K.pow(b_w_pred, 0.5)
    #b_h_pred = K.pow(b_h_pred, 0.5)
    
    loss_wh = K.sum(
        b_p * 
        (
            (K.square(b_w - b_w_pred)) + 
            (K.square(b_h - b_h_pred))
        ), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    #tensorflow.print("loss_p:", loss_p)
    #tensorflow.print("loss_xy:", loss_xy)
    #tensorflow.print("loss_wh:", loss_wh)
    #tensorflow.print("Loss:", ( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3)
    return (K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32"))/3

In [ ]:
def custom_preprocess_input(img):
    #print(img)
    #print(type(img))
    #print(img.shape)
    #plt.imshow((img).astype(np.uint8), vmin=0, vmax=255)
    #plt.show()
    #print(img.shape)
    img_pil = Image.fromarray(img.astype(np.uint8)).convert('L')
    img = np.array(img_pil)
    img = np.repeat(img[..., np.newaxis], 3, -1)
    #print(img.shape)
    #plt.imshow(img.astype(np.uint8), vmin=0, vmax=255)
    #plt.show()
    img = img.astype('float64')
    img /= 255.0
    return img

In [ ]:
BATCH_SIZE = 128
IMG_SIZE = 224

In [ ]:
train_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/train2.csv")
valid_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/val2.csv")
test_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/test2.csv")

In [ ]:
def convert_paths(path_string):
    temp = path_string.replace("\\", "/")  
    return "/data01/ML/" + temp.split("/",1)[1]

In [ ]:
train_df["image_path"] = train_df["image_path"].apply(convert_paths)
valid_df["image_path"] = valid_df["image_path"].apply(convert_paths)
test_df["image_path"] = test_df["image_path"].apply(convert_paths)

In [ ]:
train_df.head(3)

In [ ]:
train_datagen = ImageDataGenerator(
    #rescale=1./255,
    preprocessing_function = custom_preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=None,
        x_col="image_path",
        y_col=["p","x","y","w","h"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=None,
        x_col="image_path",
        y_col=["p","x","y","w","h"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

count = 0
for el in train_generator:
    count = count +1
    image = el[0][0]
    classf = el[1][0]
    #img_pil = Image.fromarray(image.astype(np.uint8))
    print(classf)
    plt.imshow(image.astype(np.uint8), vmin=0, vmax=255)
    ax = plt.gca()
    # Create a Rectangle patch
    print(classf[0])
    if classf[0]==1:
        rect = patches.Rectangle((classf[1] - int(classf[3]/2)
                          ,classf[2] - int(classf[4]/2))
                         ,classf[3],classf[4],linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)
    plt.show()
    #print(classf)
    if count == 5:
        break

In [ ]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

# Net architecture

In [ ]:
inp =  Input(shape=(IMG_SIZE,IMG_SIZE,3))
model = Conv2D(32,3, strides=(1,1), padding = "same")(inp)
model = LeakyReLU(alpha=0.1)(model)
model = MaxPooling2D(pool_size=(2, 2))(model)
model = BatchNormalization()(model)

model = Conv2D(64,3, strides=(1,1), padding = "same")(model)
model = LeakyReLU(alpha=0.1)(model)
model = MaxPooling2D(pool_size=(2, 2))(model)
model = BatchNormalization()(model)

model = Conv2D(128,3, strides=(1,1), padding = "same")(model)
model = LeakyReLU(alpha=0.1)(model)
model = MaxPooling2D(pool_size=(2, 2))(model)
model = BatchNormalization()(model)

model = Conv2D(64,3, strides=(1,1), padding = "same")(model)
model = LeakyReLU(alpha=0.1)(model)
model = MaxPooling2D(pool_size=(2, 2))(model)
model = BatchNormalization()(model)

model = Conv2D(32,3, strides=(1,1), padding = "same")(model)
model = LeakyReLU(alpha=0.1)(model)
model = MaxPooling2D(pool_size=(2, 2))(model)
model = BatchNormalization()(model)

model = Conv2D(16,3, strides=(1,1), padding = "same")(model)
model = LeakyReLU(alpha=0.1)(model)
#model = MaxPooling2D(pool_size=(2, 2))(model)

model = Flatten()(model)

#Divide
#localization = Dense(128, activation='relu')(model)
localization = Dense(4, activation='relu')(localization)

classification = Dense(64, activation='relu')(model)
classification = Dense(1, activation='relu')(classification)

model = Concatenate()([localization, classification])

In [ ]:
model = Model(inputs=[inp], outputs=model)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr = 1e-4), loss=yolf_loss, metrics=[yolf_loss])

In [ ]:
mcp_save = ModelCheckpoint('standford_face_classifier.h5', save_best_only=True, monitor='val_loss', mode='min')
#reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, mode='min')

history = model.fit_generator(generator=train_generator, epochs=150, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save])

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function = custom_preprocess_input)

test_generator = train_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=None,
        x_col="image_path",
        y_col=["p","x","y","w","h"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 

#CHANGE PARALLEL MODEL
pred=model.predict_generator(test_generator,steps=STEP_SIZE_TEST, verbose=1)

In [ ]:
print(pred[0])

In [ ]:
count = 0

for el in zip(pred,test_generator.labels,test_generator.filenames):
    count = count +1
    if el[0][0]==1:
        img = Image.open(el[2])
        img1 = ImageDraw.Draw(img)
        x1,y1 = el[0][1]-(el[0][3]/2),el[0][2]-(el[0][4]/2)
        x2,y2 = el[0][1]+(el[0][3]/2),el[0][2]+(el[0][4]/2)
        img1.rectangle([(x1,y1),(x2,y2)], outline ="red") 
        img.save("./dataset/NUOVO/results/output"+ str(count) + ".jpg")
        #img.show() 
    if count == 1000:
        break